In [1]:
import numpy as np
import pandas as pd

In [2]:
# wczytanie plikow wynikowych z dysku
df_rossmann = pd.read_excel('Wynik1.xlsx')
df_perfumeria = pd.read_excel('Wynik2.xlsx')
df_notino = pd.read_excel('Wynik3.xlsx')

In [3]:
# zmiana nazw
df_rossmann = df_rossmann.rename(columns={"Name": "Nazwa Rossmann","cena": "Rossmann_cena", "cena promo": "Rossmann_promo","url": "Rossmann_url"})
df_perfumeria = df_perfumeria.rename(columns={"Name": "Nazwa Perfumeria","cena": "Perfumeria_cena", "cena promo": "Perfumeria_promo","url": "Perfumeria_url"})
df_notino = df_notino.rename(columns={"Name": "Nazwa Notino","cena": "Notino_cena", "cena promo": "Notino_promo","url": "Notino_url"})

In [4]:
# łączenie po kodzie kreskowym (możliwe, że ten sam produkt będzie w dwóch sklepach)
df_razem = df_rossmann.merge(df_perfumeria, how = 'outer', on = 'EAN').merge(df_notino, how = 'outer', on = 'EAN')
df_razem.head(5)

,Nazwa Rossmann,EAN,Rossmann_cena,Rossmann_promo,Rossmann_url,Nazwa Perfumeria,Perfumeria_cena,Perfumeria_promo,Perfumeria_url,Nazwa Notino,Notino_cena,Notino_promo,Notino_url
0,AZZARO Fun,3351500013708,179.99,94.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CALVIN KLEIN Euphoria,88300178322,149.99,89.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BRUNO BANANI Man,3616301640738,89.99,57.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADIDAS Sport CHRG,3614225964206,164.99,109.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,STR8 Freak,5201314144953,52.99,35.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# najniższa cena
df_razem['Cena prom'] = df_razem[['Rossmann_promo', 'Perfumeria_promo', 'Notino_promo']].min(axis=1)
# przywracam index
df_razem = df_razem.reset_index(drop = True)

In [6]:
df_razem.head()

,Nazwa Rossmann,EAN,Rossmann_cena,Rossmann_promo,Rossmann_url,Nazwa Perfumeria,Perfumeria_cena,Perfumeria_promo,Perfumeria_url,Nazwa Notino,Notino_cena,Notino_promo,Notino_url,Cena prom
0,AZZARO Fun,3351500013708,179.99,94.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.99
1,CALVIN KLEIN Euphoria,88300178322,149.99,89.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.99
2,BRUNO BANANI Man,3616301640738,89.99,57.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.99
3,ADIDAS Sport CHRG,3614225964206,164.99,109.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.99
4,STR8 Freak,5201314144953,52.99,35.99,https://www.rossmann.pl/Produkt/Wody-toaletowe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.99


In [7]:
# case when, jeśli najniższa cena jest z danego sklepu, to wybieram pozostałe informacje też z niego
conditions = [
    (df_razem['Cena prom'] == df_razem['Perfumeria_promo']),
    (df_razem['Cena prom'] == df_razem['Notino_promo']),
    (df_razem['Cena prom'] == df_razem['Rossmann_promo']),
    ]

values_nazwa = [df_razem['Nazwa Perfumeria'], df_razem['Nazwa Notino'], df_razem['Nazwa Rossmann']]
values_cena = [df_razem['Perfumeria_cena'], df_razem['Notino_cena'], df_razem['Rossmann_cena']]
values_url = [df_razem['Perfumeria_url'], df_razem['Notino_url'], df_razem['Rossmann_url']]

df_razem['Cena'] = np.select(conditions, values_cena)
df_razem['Produkt'] = np.select(conditions, values_nazwa)
df_razem['url'] = np.select(conditions, values_url)

In [8]:
# wysokość promocji
df_razem['Różnica'] = df_razem['Cena'] - df_razem['Cena prom']

In [9]:
# sortowanie względem największych zniżek
df = df_razem[["Produkt", "Cena", "Cena prom", "Różnica", "url"]]
df = df.sort_values(by='Różnica', ascending = False).reset_index(drop=True)
df.head()

,Produkt,Cena,Cena prom,Różnica,url
0,Hugo Boss BOSS Bottled woda toaletowa dla mężc...,645.0,311.0,334.0,https://www.notino.pl/hugo-boss/boss-no6-bottl...
1,Chopard Love Chopard zestaw upominkowy dla kobiet,548.0,224.7,323.3,https://www.notino.pl/chopard/love-chopard-zes...
2,Jaguar Classic Black woda toaletowa dla mężczyzn,376.0,68.9,307.1,https://www.notino.pl/jaguar/classic-black-wod...
3,"Calvin Klein Euphoria Woman 100ml, 50ml i inne",485.0,200.5,284.5,https://www.notino.pl/calvin-klein/euphoria-wo...
4,Calvin Klein Euphoria woda perfumowana dla kobiet,612.0,330.2,281.8,https://www.notino.pl/calvin-klein/euphoria-wo...


In [11]:
# tworzę arkusz
with pd.ExcelWriter('Promocje.xlsx') as writer:
    df.to_excel(writer, sheet_name='Promocje', index = False, freeze_panes = [1,0])

In [12]:
# formatowanie arkusza 
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment

wb = load_workbook('Promocje.xlsx')
ws = wb['Promocje']

# formatowanie nagłówka
for cell in ws[1]:
    cell.fill = PatternFill(start_color='8FBC8F', fill_type="solid") 
    cell.alignment = Alignment(horizontal='left')
    cell.font = Font(color='FFFFFF', bold = True)
    
# szerokość pierwszej kolumny
ws.column_dimensions['A'].width = 30

# zamiana url tekstowego na hiperłącze
for i in range(0, len(df)):
    ws.cell(row=i+2, column=5).hyperlink = str(df.iloc[i,4])
    ws.cell(row=i+2, column=5).value = "Link"
    ws.cell(row=i+2, column=5).style = "Hyperlink"

# kolorowanie co drugiego wiersza
for i in range(3, len(df)+2, 2):  
    for cell in ws[i]:
        cell.fill = PatternFill(start_color='eafaea', fill_type="solid")

wb.save('Promocje.xlsx')
wb.close()

In [13]:
# mail z informacją o promocjach
import os
import win32com.client as win32
from pretty_html_table import build_table, pretty_html_table

outlook = win32.Dispatch('outlook.application')

adresy = ['...@adres.pl']

for i in adresy:
    
    #tabelka html
    df_html = build_table(df, 'blue_light',font_size = '11px'
                                  ,font_family = 'Nano Sans, sans-serif'
                                  ,text_align = 'left'
                                  ,width_dict = ['220px','auto','70px','auto', '600px']
                                  ,padding = '5px')

    # wysyłanie maila
    Msg = outlook.CreateItem(0)
    Msg.To = i
    Msg.Subject = "Promocje"
    # żeby zadziałało trzeba ustawić precyzyjną ścieżkę
    Msg.Attachments.Add(Source = os.getcwd() + '\\Promocje.xlsx')
    body = ( "<h3>Promocje</h3><br><br>" + df_html)
    Msg.HTMLBody = body
    Msg.Send()
    #Msg.display(True)